### Assignment: Code-Focused Inference

Your task is to load a pre-trained GPT-2 model and configure it to answer *only* questions related to Python coding.

1. **Load Model and Tokenizer:** Load a suitable pre-trained GPT-2 model and its corresponding tokenizer. You can use `transformers.AutoModelForCausalLM` and `transformers.AutoTokenizer`. A smaller model like `gpt2` or `gpt2-medium` might be sufficient.
2. **Implement a Filtering Mechanism:** Before generating a response, check if the input prompt is related to Python coding. You can use simple keyword matching (e.g., "Python", "code", "function", "class", "import") or a more sophisticated approach using a text classification model (optional).
3. **Generate Response:** If the prompt is deemed a Python coding question, generate a response using the loaded GPT-2 model.
4. **Handle Non-Coding Questions:** If the prompt is not related to Python coding, return a predefined message indicating that the model can only answer coding questions.
5. **Test:** Test your implementation with various prompts, including both Python coding questions and non-coding questions, to ensure the filtering mechanism works correctly.

# 1. Load Model and Tokenizer

 Load a suitable pre-trained GPT-2 model and its corresponding tokenizer. You can use `transformers.AutoModelForCausalLM` and `transformers.AutoTokenizer`. A smaller model like `gpt2` or `gpt2-medium` might be sufficient.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained GPT-2 model
model = AutoModelForCausalLM.from_pretrained('gpt2')

# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2. **Implement a Filtering Mechanism:**

Before generating a response, check if the input prompt is related to Python coding. You can use simple keyword matching (e.g., "Python", "code", "function", "class", "import") or a more sophisticated approach using a text classification model (optional)

In [2]:
def is_python_coding_question(prompt):
    """
    Checks if the input prompt is related to Python coding using keyword matching.

    Args:
        prompt: The input string.

    Returns:
        True if the prompt contains Python coding keywords, False otherwise.
    """
    python_keywords = [
        "python", "code", "function", "class", "import", "def", "print",
        "loop", "list", "dictionary", "numpy", "pandas", "matplotlib",
        "sklearn", "tensorflow", "pytorch", "def", "return", "if", "else",
        "elif", "for", "while", "try", "except", "finally", "with", "as"
    ]
    prompt_lower = prompt.lower()
    for keyword in python_keywords:
        if keyword in prompt_lower:
            return True
    return False

3. **Generate Response:** If the prompt is deemed a Python coding question, generate a response using the loaded GPT-2 model.

In [4]:
import torch

def answer_coding_question(prompt, model, tokenizer, max_len=100):
    """
    Decides whether to generate a response using the GPT-2 model or return a
    predefined message based on whether the prompt is a Python coding question.

    Args:
        prompt: The input string.
        model: The pre-trained GPT-2 model.
        tokenizer: The corresponding tokenizer.
        max_len: The maximum length of the generated response.

    Returns:
        The generated response if the prompt is a coding question,
        otherwise a predefined message.
    """
    if is_python_coding_question(prompt):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Ensure pad token is set before tokenization
        if tokenizer.pad_token is None:
             tokenizer.pad_token = tokenizer.eos_token

        inputs = tokenizer(
            prompt,
            return_tensors='pt',
            padding=True,
            return_attention_mask=True
        )
        inputs = {key: value.to(device) for key, value in inputs.items()}


        generated_ids = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_len,
            pad_token_id=tokenizer.pad_token_id
        )
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        return generated_text
    else:
        return "I can only answer questions related to Python coding."

# Validations on python - non-python coding questions:



In [5]:
# 1. Call with a Python coding question
prompt_coding_1 = "How to define a function in Python?"
response_coding_1 = answer_coding_question(prompt_coding_1, model, tokenizer)
print(f"Prompt: {prompt_coding_1}")
print(f"Response: {response_coding_1}\n")

Prompt: How to define a function in Python?
Response: How to define a function in Python?

The following code snippet shows how to define a function in Python.

import sys import time import sys.argv import sys.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.argv.



In [6]:
# 2. Call with a non-coding question
prompt_non_coding_1 = "What is the capital of France?"
response_non_coding_1 = answer_coding_question(prompt_non_coding_1, model, tokenizer)
print(f"Prompt: {prompt_non_coding_1}")
print(f"Response: {response_non_coding_1}\n")

Prompt: What is the capital of France?
Response: I can only answer questions related to Python coding.



In [7]:
# 3. Call with another Python coding question
prompt_coding_2 = "Explain Python lists."
response_coding_2 = answer_coding_question(prompt_coding_2, model, tokenizer)
print(f"Prompt: {prompt_coding_2}")
print(f"Response: {response_coding_2}\n")

Prompt: Explain Python lists.
Response: Explain Python lists.

The list is a list of Python objects that are used to represent the Python objects in the list.

The list is a list of Python objects that are used to represent the Python objects in the list. The list is a list of Python objects that are used to represent the Python objects in the list. The list is a list of Python objects that are used to represent the Python objects in the list. The list is a list of Python objects that are used



In [8]:
# 4. Call with another non-coding question
prompt_non_coding_2 = "Tell me a joke."
response_non_coding_2 = answer_coding_question(prompt_non_coding_2, model, tokenizer)
print(f"Prompt: {prompt_non_coding_2}")
print(f"Response: {response_non_coding_2}\n")

Prompt: Tell me a joke.
Response: I can only answer questions related to Python coding.

